In [ ]:
import os
import sys
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from model_settings import ms
from feature_collector import preprocess_data
from df_collector import df_collector
root = Path().resolve().parent.parent
df_collector.root = root

In [ ]:
from convsklearn import barrier_trainer, asian_trainer
price_name = 'barrier_price' 
filetag = 'cboe barriers'
trainer = barrier_trainer

dataset = df_collector.cboe_spx_barriers().iloc[:,1:]
dataset = dataset[dataset[price_name]<dataset['spot_price']].dropna().reset_index(drop=True).copy()
dates = pd.Series(np.sort(dataset['date'].unique()))

In [ ]:
"""
define train test arrays!
"""

In [ ]:
dataset['moneyness'] = ms.df_moneyness(dataset)
dataset['m'] = dataset['moneyness']/dataset['spot_price']
dataset['b'] = dataset['barrier']/dataset['spot_price']

In [ ]:
train_dates = dates.iloc[:len(dates)//4]
test_dates = dates[~dates.isin(train_dates)]

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neural_network import MLPRegressor

In [ ]:
feature_set = [
    'm',
    'b',
    'days_to_maturity',
    'risk_free_rate',
    'dividend_rate',
    'kappa',
    'theta',
    'rho',
    'eta',
    'v0',
    'barrier',
    'barrier_type_name',
    'w'
]
feature_set

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('categorical',OneHotEncoder(),['w','averaging_type'])
],remainder='passthrough',force_int_remainder_cols=False)

dnn_pipeline = make_pipeline(preprocessor,MLPRegressor(random_state=1312))
dnn = dnn_pipeline.fit(train_X,train_y)

In [ ]:
train_data = dataset[dataset['date'].isin(train_dates)]
train_X = train_data[feature_set]
train_y = train_data['observed_price']

In [ ]:
test_data = dataset[dataset['date'].isin(test_dates)]
test_X = test_data[feature_set]
train_y = test_data['observed_price']
np.mean(np.abs(dnn.predict(test_X)-train_y))